In [1]:
!hostname #main container name

a660af172807


# Part 1

In [1]:
!wget https://pages.cs.wisc.edu/~harter/cs639/data/hdma-wi-2021.csv

--2023-03-31 02:40:08--  https://pages.cs.wisc.edu/~harter/cs639/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv’

hdma-wi-2021.csv    100%[===================>] 166.84M  8.26MB/s    in 25s     

2023-03-31 02:40:33 (6.73 MB/s) - ‘hdma-wi-2021.csv’ saved [174944099/174944099]



In [5]:
!hdfs dfs -D dfs.block.size=1M -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://main:9000/single.csv

In [6]:
!hdfs dfs -D dfs.block.size=1M -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://main:9000/double.csv

In [7]:
!hdfs dfs -du -h hdfs://main:9000/

166.8 M  333.7 M  hdfs://main:9000/double.csv
166.8 M  166.8 M  hdfs://main:9000/single.csv


# Part 2

In [8]:
import requests

In [10]:
d_blocks={}
x=0
while x<174692761: # up to 167 MBs in bytes
    req = requests.get(f'http://main:9870/webhdfs/v1/single.csv?op=OPEN&offset={x}', allow_redirects=False)
    if req.headers["Location"][7:46] in d_blocks:
        d_blocks[req.headers["Location"][7:46]]=d_blocks[req.headers["Location"][7:46]] + 1
    else: 
        d_blocks[req.headers["Location"][7:46]]=1
    x+=1048576

d_blocks

{'09d1c50c76cf:9864/webhdfs/v1/single.csv': 94,
 '8f93d9669d45:9864/webhdfs/v1/single.csv': 73}

# Part 3

In [11]:
import io, time

In [12]:
class hdfsFile(io.RawIOBase):
    def __init__(self, path):
        self.path = path
        self.offset = 0 #bytes I read
        
        res = requests.get(f'http://main:9870/webhdfs/v1/{self.path}?op=GETFILESTATUS', allow_redirects=False)
        self.length = res.json()['FileStatus']['length']

        
    def readable(self):
        return True

    
    def readinto(self, b):
        if self.offset >= self.length:
            return 0
        
        bytes_todo = min(len(b), self.length - self.offset)
        
        url = f'http://main:9870/webhdfs/v1/{self.path}'
        ress = requests.get(url=url, params={"op": "OPEN", "offset": self.offset, "length": bytes_todo})
        
        if ress.status_code != 200:
            b[0:1] = b"\n"
            self.offset = (self.offset // 1024 + 1) * 1024  #start of next block
            return 1
        
        b[0:bytes_todo] = bytearray(ress.content)
        self.offset += bytes_todo

        return bytes_todo

In [13]:
#size 1
Single_fam=0
Multi_fam=0
t0 = time.time()

for line in io.BufferedReader(hdfsFile("single.csv"), buffer_size=100000):
    line = str(line, "utf-8")
    #print(line)
    if "Single Family" in line:
        Single_fam+=1
    if "Multifamily" in line:
        Multi_fam+=1
    
end=time.time()-t0
print("Counts from single.csv", "\nSingle Family:", Single_fam, "\nMulti Family:", Multi_fam, "\nSeconds:", end)

Counts from single.csv 
Single Family: 444874 
Multi Family: 2493 
Seconds: 42.1686737537384


In [14]:
#size 2
Single_fam=0
Multi_fam=0
t0 = time.time()

for line in io.BufferedReader(hdfsFile("single.csv"), buffer_size=1024**2):
    line = str(line, "utf-8")
    #print(line)
    if "Single Family" in line:
        Single_fam+=1
    if "Multifamily" in line:
        Multi_fam+=1
    
end=time.time()-t0

print("Counts from single.csv", "\nSingle Family:", Single_fam, "\nMulti Family:", Multi_fam, "\nSeconds:", end)

Counts from single.csv 
Single Family: 444874 
Multi Family: 2493 
Seconds: 6.35726261138916


## Part 4

In [ ]:
!hdfs dfsadmin -fs hdfs://main:9000/ -report  #run after docker kill, I did worker 2

In [ ]:
Single_fam=0
Multi_fam=0

for line in io.BufferedReader(hdfsFile("single.csv"), buffer_size=1024**2):
    line = str(line, "utf-8")
    #print(line)
    if "Single Family" in line:
        Single_fam+=1
    if "Multifamily" in line:
        Multi_fam+=1

print("Counts from single.csv", "\nSingle Family:", Single_fam, "\nMulti Family:", Multi_fam)

In [ ]:
Single_fam=0
Multi_fam=0

for line in io.BufferedReader(hdfsFile("double.csv"), buffer_size=1024**2):
    line = str(line, "utf-8")
    #print(line)
    if "Single Family" in line:
        Single_fam+=1
    if "Multifamily" in line:
        Multi_fam+=1
    

print("Counts from double.csv", "\nSingle Family:", Single_fam, "\nMulti Family:", Multi_fam)